In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#do not use these variable names: f0, f1, f2, g1, g2, g3, g5, f_new1, f_new2, f_new3

from iHMM_GP.step1_utils import *
from iHMM_GP.data_setup import *

/Users/sunrit/opt/anaconda3/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:1058: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)


# DEMO 2: EFFECT OF M

In [2]:
def estimate(data_train, data_test3, gp_fitting_results, M, s_train_true, s_test_true):
    import time
    iHMM_params = (1, 1, 1)
    n_jobs= -1; N_max=1000
    
    T = len(data_train)
    if M < T:
        min_cluster_size = 5
    else:
        min_cluster_size = 5
        
    x, y = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
    Z = np.array([[x[i, j], y[i, j]] for i in range(10) for j in range(10)])
    
    marginal_lls, sigma2, all_kernels = gp_fitting_results
    
    time0 = time.time()
    if M < T:
        # get block results
        results = get_block_results(
                data_train, marginal_lls, all_kernels, sigma2, iHMM_params, Z, M, min_cluster_size=min_cluster_size, verbose=False)
        K_opt, means = get_K(results, high=5, Z=Z, plot=False)
        s1 = get_final_states(K_opt, means, results)
    else:
        s1, o1, models1, N1, M1 = forward_pass(data_train, iHMM_params, marginal_lls, all_kernels, sigma2, Z)
        s1, _, _, _, _ = refinement(s1, o1, N1, M1, models1, iHMM_params, sigma2, marginal_lls, data_train, min_cluster_size=min_cluster_size, verbose=False)
    
    
    # use block results to get models
    final_models_ = get_final_results(data_train, s1, sigma2, Z, n_jobs=n_jobs, N_max=N_max)

    # compute likelihoods of each observed points in data_train
    logliks_train = get_likelihoods(data_train, final_models_, sigma2); logliks_train = np.array(logliks_train)


    # viterbi for train and test3
    Pi_est, _ = get_nm(s1, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    s_final_train = viterbi(None, None, Pi_hat, logliks_train)
    
    s_final_train = rename_s(s_final_train)[1]
    K_opt = len(np.unique(s_final_train))
            
    final_models_ = get_final_results(data_train, s_final_train, sigma2, Z, n_jobs=n_jobs, N_max=N_max) #new 2 lines
    time0 = time.time() - time0
    
    logliks_train = get_likelihoods(data_train, final_models_, sigma2); logliks_train = np.array(logliks_train) #
    Pi_est, _ = get_nm(s_final_train, np.zeros(T)); row_sums = Pi_est.sum(axis=1)
    Pi_hat = Pi_est / row_sums[:, np.newaxis]
    
    logliks_test = get_likelihoods(data_test3, final_models_, sigma2); logliks_test = np.array(logliks_test)
    s_final_test = viterbi(None, None, Pi_hat, logliks_test)
    
    hmm_loglik_train = compute_likelihood_HMM(Pi_hat, logliks_train)
    hmm_loglik_test = compute_likelihood_HMM(Pi_hat, logliks_test) # computed on test3

    from sklearn.metrics import rand_score, adjusted_mutual_info_score, v_measure_score # (rand, NMI, vscore)
    train_label_acc = (rand_score(s_train_true, s_final_train), adjusted_mutual_info_score(s_train_true, s_final_train), v_measure_score(s_train_true, s_final_train))
    test_label_acc = (rand_score(s_test_true, s_final_test), adjusted_mutual_info_score(s_test_true, s_final_test), v_measure_score(s_test_true, s_final_test))
    
    return {'K': K_opt,
           'train_label_accuracy': train_label_acc,
            'test_label_accuracy': test_label_acc,
           'time': time0,
           'hmm_loglik_train': hmm_loglik_train,
            'hmm_loglik_test': hmm_loglik_test}
    

In [3]:
def demo2_2d(T=600, n=50, sigma2=2):
    T_test = 100

    # create data
    data, s = sim_new_data2(f_true, Pi_true, T=T+T_test, n=n, sigma2=sigma2)

    # get training data - for t in tau, hold out q% of the data
    data_train = []

    for t in range(T):
        X, Y = data[t]
        data_train.append((X, Y))

    # test 3 - new sequence state prediction
    data_test3 = data[T:]

    # state labels:
    s_train_true = s[0:T]
    s_test_true = s[T:]
    
    import time
    time0 = time.time()
    marginal_lls, sigma2, all_kernels = get_all_GPs(data_train)
    time0 = time.time() - time0
    print('done GP')
    
    # 50
    res50 = estimate(data_train, data_test3, (marginal_lls, sigma2, all_kernels), M=50, s_train_true=s_train_true, s_test_true=s_test_true)
    res50['time'] = res50['time'] + time0
    print('done 50')
    
    # 100
    res100 = estimate(data_train, data_test3, (marginal_lls, sigma2, all_kernels), M=100, s_train_true=s_train_true, s_test_true=s_test_true)
    res100['time'] = res100['time'] + time0
    print('done 100')
    
    # 300
    res300 = estimate(data_train, data_test3, (marginal_lls, sigma2, all_kernels), M=300, s_train_true=s_train_true, s_test_true=s_test_true)
    res300['time'] = res300['time'] + time0
    print('done 300')
    
    # full, 600
    res = estimate(data_train, data_test3, (marginal_lls, sigma2, all_kernels), M=T, s_train_true=s_train_true, s_test_true=s_test_true)
    res['time'] = res['time'] + time0
    print('done 600')
    
    return (res50, res100, res300, res)

In [4]:
# run on local machine
import time

time0 = time.time()
results = demo2_2d(T=600, n=50, sigma2=2)
time0 = time.time() - time0

print('total time taken:', np.round(time0, 2))
results

done GP
done 50
done 100
done 300
done 600
total time taken: 705.72


({'K': 7,
  'train_label_accuracy': (0.8923539232053422,
   0.6919234839007531,
   0.6982429236583766),
  'test_label_accuracy': (0.9236363636363636,
   0.825473066615895,
   0.8496456575053438),
  'time': 110.43455290794373,
  'hmm_loglik_train': -107669.61190404126,
  'hmm_loglik_test': -18020.69956010824},
 {'K': 12,
  'train_label_accuracy': (0.9303338898163606,
   0.7568762167456529,
   0.7653316257896411),
  'test_label_accuracy': (0.9664646464646465,
   0.8728981463226998,
   0.8972675142455011),
  'time': 163.19627809524536,
  'hmm_loglik_train': -106464.36475127794,
  'hmm_loglik_test': -17861.120457657147},
 {'K': 10,
  'train_label_accuracy': (0.9692487479131886,
   0.9123699676384763,
   0.9148241885383248),
  'test_label_accuracy': (0.9913131313131314,
   0.9627490555679613,
   0.9700022692315299),
  'time': 167.1029829978943,
  'hmm_loglik_train': -105884.39942662008,
  'hmm_loglik_test': -17773.52804715076},
 {'K': 9,
  'train_label_accuracy': (1.0, 1.0, 1.0),
  'test_la

In [27]:
# run on cluster with 20 cores
import pickle
from tqdm import tqdm
results = []

for rep in range(20):
    print('working rep ', rep)
    res = demo2_2d(T=600, n=50, sigma2=2)
    results.append(res)
    
pickle.dump(results, open( "new_simulation_results/demo2.p", "wbeffect" ) )

working rep  0
done GP


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


done 50
done 100
done 100
done 300
working rep  1
done GP


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


done 50
done 100
done 100


...

done 50
done 100
done 100


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


done 300
working rep  19
done GP
done 50


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


done 100
done 100
done 300
